In [ ]:
import geopandas  as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h3 as h3
import osmnx as ox
import contextily as ctx

import itertools
from operator import itemgetter


from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString

import re

import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

path =  Path(os.getcwd())
root = path.parent.absolute()

root

In [ ]:
gdf = gpd.read_file(root / 'X.data' / 'raw_data' / 'la_county_website_data' /  'LA_County_City_Boundaries'/ 'LA_County_City_Boundaries.shp')
gdf.sample(5)

mask = gdf.CITY_LABEL.isin(['Long Beach', 'Signal Hill'])

gdf_lb = gdf[mask]
gdf_lb.sample(2)

In [ ]:
crash_dir = root / 'X.data' / 'raw_data'/ 'TIMS_raw_crashes_downloads'

#print(join(crash_dir, 'Crashes_2014.csv'))
onlyfiles = [f for f in listdir(crash_dir) if isfile(crash_dir / f)]

appended_data = []

for file in onlyfiles:
    print(file)
    temp = pd.read_csv(crash_dir / file, low_memory = False)
    appended_data.append(temp)

collision_df = pd.concat(appended_data)
collision_df.columns

In [ ]:
print(collision_df.shape)
#print(len(collision_df.CASE_ID.unique()))
collision_df = collision_df[['ACCIDENT_YEAR', 'COLLISION_DATE', 'COLLISION_TIME', 'DAY_OF_WEEK', 'CITY', 'POINT_X', 'POINT_Y']]
collision_df.drop_duplicates(inplace = True)
collision_df = collision_df.reset_index(drop = True)
collision_df = collision_df.reset_index()
collision_df = collision_df.rename(columns = {'index':'collision_id'})

print(collision_df.shape)
collision_df.sample(3)

In [ ]:
coll_lb_18 = collision_df.query('ACCIDENT_YEAR == 2018 and CITY == "LONG BEACH"')
coll_lb_18.sample()

from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(coll_lb_18.POINT_X, coll_lb_18.POINT_Y)]
gdf_coll_lb = GeoDataFrame(coll_lb_18, geometry=geometry)

gdf_coll_lb.set_crs(epsg=4326, inplace=True).sample(2)

gdf_coll_lb.sample(2)

In [ ]:
edges = gpd.read_file(root / 'X.data' / 'semi_processed'/'nodes_and_edges' / 'la_county_edges' / 'la_county_edges.shp')
edges = edges.to_crs("EPSG:3857")
edges_motorway = edges[edges['highway'] == 'motorway']
display(edges_motorway.sample(2))
print(edges.shape)

In [ ]:
maxBuff = 75
lines = edges_motorway.copy()
points = gdf_coll_lb.to_crs(epsg=3857).copy()

points['geometry'] = points.geometry.buffer(maxBuff)
gdf_coll_lb_fwy = gpd.sjoin(lines[['geometry']], points.to_crs(epsg=3857), how = 'inner', op ='intersects')
gdf_coll_lb_fwy = gdf_coll_lb_fwy.drop(columns = ['geometry'])
gdf_coll_lb_fwy = gdf_coll_lb_fwy.drop_duplicates()

gdf_coll_lb_fwy = gpd.GeoDataFrame(
    gdf_coll_lb_fwy, geometry=gpd.points_from_xy(gdf_coll_lb_fwy.POINT_X, 
                                                 gdf_coll_lb_fwy.POINT_Y,
                                                 crs="EPSG:4326"))

#display(gdf_coll_lb_fwy.sample(2))
gdf_coll_lb_fwy.shape

In [ ]:
gdf_coll_lb_fwy.sample(2)

In [ ]:
def ckdnearest_two_dist(points_frame, gdf_ref):

    nA = points_frame[['point_x', 'point_y']].to_numpy()
    #print(nA)
    nB = np.array(list(gdf_ref.geometry.apply(lambda x: (x.x, x.y))))
    #print(nB)
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=2)
    dist1 = [item[0] for item in dist]
    dist2 = [item[1] for item in dist]
    idx1 = [item[0] for item in idx]
    idx2 = [item[1] for item in idx]
    #print(dist1[0:2])
    #print(dist2[0:2])
    gdB_nearest1 = gdf_ref.iloc[idx1][['geometry']].reset_index(drop=True)
    gdB_nearest1.columns = ['geometry_1']
    gdB_nearest2 = gdf_ref.iloc[idx2][['geometry']].reset_index(drop=True)
    gdB_nearest2.columns = ['geometry_2']
    gdf = pd.concat(
        [
            points_frame[['point_id']].reset_index(drop=True),
            gdB_nearest1,
            gdB_nearest2,
            pd.Series(dist1, name='exit1_distance'),
            pd.Series(dist2, name='exit2_distance')
        ], 
        axis=1)
    
    return gdf

In [ ]:
gdf_exits = gpd.read_file(root / 'X.data' / 'raw_data' / 'la_county_website_data' /  'Freeway_Exits'/ 'Freeway_Exits.shp')
gdf_exits = gdf_exits.to_crs("EPSG:3857")

gdf_exits_lb = clip(gdf_exits, gdf_lb)
gdf_exits_lb.sample(2)

In [ ]:
edge_exploded = edges_motorway.explode('geometry', index_parts=False)
edge_exploded = pd.DataFrame(edge_exploded)
print(edge_exploded.shape)
points_list = edge_exploded.geometry.apply(lambda geom: list(geom.coords))

edge_exploded['points_list'] = points_list

edge_motorway_points = edge_exploded.explode('points_list')
edge_motorway_points = edge_motorway_points.drop(columns=['osmid', 'geometry'])
                                                          
edge_motorway_points = edge_motorway_points.reset_index(drop = True)
edge_motorway_points = edge_motorway_points.reset_index()
edge_motorway_points = edge_motorway_points.rename(columns = {'index':'point_id'})  

edge_motorway_points[['point_x', 'point_y']] = pd.DataFrame(edge_motorway_points['points_list'].tolist(), index=edge_motorway_points.index)


In [ ]:
edge_motorway_points.sample(2)

In [ ]:
edge_exits_distance = ckdnearest_two_dist(edge_motorway_points, gdf_exits_lb)
edge_exits_distance.sample()

In [ ]:
edge_motorway_points = edge_motorway_points.merge(edge_exits_distance[['point_id', 'exit1_distance', 'exit2_distance']], on = 'point_id', how = 'left')

In [ ]:
edge_motorway_points = gpd.GeoDataFrame(
    edge_motorway_points, geometry=gpd.points_from_xy(edge_motorway_points.point_x, 
                                                      edge_motorway_points.point_y,
                                                      crs="EPSG:3857"))
edge_motorway_points.sample(2)

In [ ]:
edge_motorway_points.sample(2)

In [ ]:
base = gdf_lb.plot(markersize=0.01, alpha = .2, edgecolors= "black", figsize=(23, 30))
edge_motorway_points.plot('exit2_distance', ax=base, marker = 'o', markersize = 24, zorder=1)
gdf_coll_lb_fwy.to_crs(epsg=3857).plot(ax=base, marker = 'o', markersize = 24, color = 'red', zorder=3)


gdf_exits_lb.to_crs(epsg=3857).plot(ax=base, marker='o', color='yellow', markersize=12, zorder=2)

ctx.add_basemap(base)
plt.show()

In [ ]:
gdf_coll_lb_fwy.head(1)

In [ ]:
edge_coord_df_4326 = edge_motorway_points.copy()
edge_coord_df_4326.crs = "EPSG:3857"
edge_coord_df_4326 = edge_coord_df_4326.to_crs({'init': 'epsg:4326'}) 
edge_coord_df_4326.head(1)

In [ ]:
print(edge_coord_df_4326.shape)
sum(edge_coord_df_4326.name.value_counts())

In [ ]:
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="name"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["geometry"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [ ]:
print(gdf_coll_lb_fwy.shape)
gdf_coll_lb_fwy["name"] = gdf_coll_lb_fwy.apply(get_nearest_values, other_gdf=edge_coord_df_4326, point_column="geometry", axis=1)


In [ ]:
gdf_coll_lb_fwy.sample(2)

In [ ]:
print(sum(gdf_coll_lb_fwy.name.value_counts()))

In [ ]:
import numpy as np
from scipy.spatial import KDTree
rng = np.random.default_rng()

gdf_ref = gdf_coll_lb_fwy.copy()
points_frame = edge_coord_df_4326.copy()


def neighbor_counts(ref_data, row, circle_radius, accident_year):
    #print(row.ref)
    #print(row)
    count = 0
    ref_data = ref_data[ref_data['name'] == row['name']]
    ref_data = ref_data[ref_data['ACCIDENT_YEAR'] == accident_year]
    
    if ref_data.shape[0] > 0:
        nA = ref_data[['POINT_X', 'POINT_Y']].to_numpy()
        #print(nA)
        nB = []
        nB.append(list([row.geometry.x, row.geometry.y]))
        #print(nB)
        #print(nB)
        kd_collisions = KDTree(nA)
        kd_points = KDTree(nB)

        count = kd_points.count_neighbors(kd_collisions,r=circle_radius)
    return count

edge_motorway_points['coll_2018_radius_0250'] = edge_coord_df_4326.apply(
    lambda row: neighbor_counts(gdf_coll_lb_fwy, row, circle_radius = .0025, accident_year = 2018),
    axis=1
)

edge_motorway_points['coll_2018_radius_0500'] = edge_coord_df_4326.apply(
    lambda row: neighbor_counts(gdf_coll_lb_fwy, row, circle_radius = .0050, accident_year = 2018),
    axis=1
)

edge_motorway_points['coll_2018_radius_0750'] = edge_coord_df_4326.apply(
    lambda row: neighbor_counts(gdf_coll_lb_fwy, row, circle_radius = .0075, accident_year = 2018),
    axis=1
)

edge_motorway_points['coll_2018_radius_1000'] = edge_coord_df_4326.apply(
    lambda row: neighbor_counts(gdf_coll_lb_fwy, row, circle_radius = .0100, accident_year = 2018),
    axis=1
)

edge_motorway_points['coll_2018_radius_1000'] = edge_coord_df_4326.apply(
    lambda row: neighbor_counts(gdf_coll_lb_fwy, row, circle_radius = .0150, accident_year = 2018),
    axis=1
)


In [ ]:
edge_motorway_points.coll_2018_radius_1000.value_counts()

In [ ]:
edge_motorway_points.coll_2018_radius_0750.value_counts()

In [ ]:
edge_motorway_points.coll_2018_radius_0500.value_counts()

In [ ]:
edge_motorway_points.coll_2018_radius_0250.value_counts()

In [ ]:
base = gdf_lb.plot(markersize=0.01, alpha = .2, edgecolors= "black", figsize=(23, 30))
#edge_coord_df_4326.to_crs(epsg=3857).head(1).plot(ax=base, marker = 'o', markersize = 500, color = 'purple', zorder=1)
edge_motorway_points.plot('coll_2018_radius_0250', ax=base, marker = 'o', markersize = 24, zorder=3)
#gdf_coll_lb_fwy.to_crs(epsg=3857).plot(ax=base, alpha = .25, marker = 'o', markersize = 24, color = 'red', zorder=2)
#df_filtered.to_crs(epsg=3857).plot(ax=base, marker = 'o', markersize = 200, color = 'orange', zorder=3)

gdf_exits_lb.to_crs(epsg=3857).plot(ax=base, alpha = .75, marker='o', color='yellow', markersize=12, zorder=2)

ctx.add_basemap(base)
plt.show()

In [ ]:
volume = gpd.read_file(root / 'X.data' / 'raw_data'/ 'Traffic_Volumes_AADT' / 'HWY_Traffic_Volumes_AADT.shp')
volume.sample(2)

In [ ]:
print(volume.shape)
print(len(volume.OBJECTID.unique()))
volume_exploded = volume.explode('geometry', index_parts=False)
volume_exploded = pd.DataFrame(volume_exploded)
print(volume_exploded.shape)

In [ ]:
volume_exploded = volume_exploded.groupby('OBJECTID').first()
volume_exploded = volume_exploded.reset_index(drop = True)
volume_exploded = volume_exploded.reset_index()
volume_exploded = volume_exploded.rename(columns = {'index':'volume_id'}) 
volume_exploded.shape

In [ ]:
volume_exploded.ROUTE.unique()
volume_exploded['ROUTE'] = volume_exploded['ROUTE'].map(str)

In [ ]:
edge_motorway_points['ref_num'] = edge_motorway_points['ref'].str.replace(r'\D', '')
edges_motorway_list = edge_motorway_points['ref_num'].unique()
volume_exploded_lb = volume_exploded[volume_exploded['ROUTE'].isin(edges_motorway_list)]
volume_exploded_lb.shape

In [ ]:
display(volume_exploded_lb)

In [ ]:
hwy_ref = '710'

edge_filtered = edge_motorway_points[edge_motorway_points['ref_num'] == hwy_ref]
volume_filtered = volume_exploded_lb[volume_exploded_lb['ROUTE'] == hwy_ref]

print(edge_filtered.shape)

ckdnearest_volume_interpolation(edge_filtered.head(27).tail(1), volume_filtered)

In [ ]:
def ckdnearest_volume_interpolation(points_frame, gdf_ref):
    
    gdf_ref = gdf_ref.reset_index(drop = True)
    
    nA = points_frame[['point_x', 'point_y']].to_numpy()
    #print(nA)
    nB = np.array(list(gdf_ref.geometry.apply(lambda x: (x.x, x.y))))
    #print(nB)
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=2)
    dist1 = [item[0] for item in dist]
    dist2 = [item[1] for item in dist]
    idx1 = [item[0] for item in idx]
    idx2 = [item[1] for item in idx]
    #print(idx)
    #print(dist)
    
    volume_cols = ['PM','BACK_PEAK_','BACK_AADT','AHEAD_PEAK','AHEAD_AADT']
    
    gdB_nearest1 = gdf_ref.iloc[idx1][volume_cols].reset_index(drop=True)
    gdB_nearest1 = gdB_nearest1.add_suffix('_1')
    gdB_nearest2 = gdf_ref.iloc[idx2][volume_cols].reset_index(drop=True)
    gdB_nearest2 = gdB_nearest2.add_suffix('_2')
    #print(points_frame[['point_id']])

    
    gdf = pd.concat(
        [
            points_frame[['point_id']].reset_index(drop=True),
            gdB_nearest1,
            gdB_nearest2,
            pd.Series(dist1, name='dist_1'),
            pd.Series(dist2, name='dist_2')
        ], 
        axis=1)
    
    dist2_mask = gdf['dist_1']<gdf['dist_2']
    
    gdf['VOLUME_REF_DISTANCE'] = gdf['dist_1']
    gdf[dist2_mask]['EXIT_DIST'] = 'dist_2'
    
    gdf['PM'] = gdf['PM_1']
    gdf[dist2_mask]['PM'] = 'PM_2'
     
    gdf['BACK_PEAK'] = gdf['BACK_PEAK__1']
    gdf[dist2_mask]['BACK_PEAK'] = 'BACK_PEAK__2'
    
    gdf['BACK_AADT'] = gdf['BACK_AADT_1']
    gdf[dist2_mask]['BACK_AADT'] = 'BACK_AADT_2'
    
    gdf['AHEAD_PEAK'] = gdf['AHEAD_PEAK_1']
    gdf[dist2_mask]['AHEAD_PEAK'] = 'AHEAD_PEAK_2'
    
    gdf['AHEAD_AADT'] = gdf['AHEAD_AADT_1']
    gdf[dist2_mask]['AHEAD_AADT'] = 'AHEAD_AADT_2'
    
    gdf['PEAK'] = gdf[['AHEAD_PEAK', 'BACK_PEAK']].mean(axis = 1)
    gdf['AADT'] = gdf[['AHEAD_AADT', 'BACK_AADT']].mean(axis = 1)
    
    gdf = gdf.drop(columns = ['dist_1', 'dist_2', 
                              'PM_1','BACK_PEAK__1','BACK_AADT_1','AHEAD_PEAK_1','AHEAD_AADT_1',
                              'PM_2','BACK_PEAK__2','BACK_AADT_2','AHEAD_PEAK_2','AHEAD_AADT_2',
                             'AHEAD_PEAK', 'BACK_PEAK', 'AHEAD_AADT', 'BACK_AADT'])
    
    return gdf

In [ ]:
hwy_vol_list = []
for hwy_ref in edges_motorway_list:
    edge_filtered = edge_motorway_points[edge_motorway_points['ref_num'] == hwy_ref]
    volume_filtered = volume_exploded_lb[volume_exploded_lb['ROUTE'] == hwy_ref]
    #print(volume_filtered.shape)
    if volume_filtered.shape[0] > 0:
        output_df = ckdnearest_volume_interpolation(edge_filtered, volume_filtered)
        hwy_vol_list.append(output_df)
#output_df

In [ ]:
#print(hwy_vol_list)
volume_ref = pd.concat(hwy_vol_list)

edge_motorway_points = edge_motorway_points.merge(volume_ref, on = 'point_id', how = 'left')
edge_motorway_points.sample(2)

In [ ]:
volume_exploded_lb = gpd.GeoDataFrame(volume_exploded_lb, geometry = volume_exploded_lb.geometry)

In [ ]:
base = gdf_lb.plot(markersize=0.01, alpha = .2, edgecolors= "black", figsize=(23, 30))
#edge_coord_df_4326.to_crs(epsg=3857).head(1).plot(ax=base, marker = 'o', markersize = 500, color = 'purple', zorder=1)
edge_motorway_points.plot('PM', ax=base, marker = 'o', markersize = 24, zorder=1)
#gdf_coll_lb_fwy.to_crs(epsg=3857).plot(ax=base, marker = 'o', markersize = 24, color = 'red', zorder=3)
volume_exploded_lb.plot(ax=base, marker = 'o', markersize = 24, color = 'red', zorder=3)

#gdf_exits_lb.to_crs(epsg=3857).plot(ax=base, marker='o', color='yellow', markersize=12, zorder=2)

ctx.add_basemap(base)
plt.show()